In [1]:
!pip install transformers
!pip install sentencepiece
!pip install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 84.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 87.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 32.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import csv
from csv import reader
from sklearn.model_selection import train_test_split

# Load the Albanian summarization dataset
#dataset = load_dataset('csv', data_files={'train': 'train.csv', 'validation': 'validation.csv'})

allNews = []
with open('/content/final.csv', 'r') as read_obj:
    # pass the file object to reader() to get the reader object
    csv_reader = reader(read_obj)
    # Iterate over each row in the csv using reader object
    for row in csv_reader:
        # row variable is a list that represents a row in csv
        allNews.append(row)

allNews.pop(0)
articleDataset = []
summaryDataset = []
for news in allNews:
  articleDataset.append(news[4])
  summaryDataset.append(news[2] + news[3])

# Split the data into training and test sets
train_article, test_article = train_test_split(articleDataset, test_size=0.2, random_state=42)
train_summary, test_summary = train_test_split(summaryDataset, test_size=0.2, random_state=42)
train_article, validation_article, train_summary, validation_summary = train_test_split(train_article, train_summary, test_size=0.2, random_state=42)


In [3]:
import torch
from transformers import BartTokenizer, BartForConditionalGeneration, AdamW
import csv
from csv import reader
from sklearn.model_selection import train_test_split


# Load the pre-trained BART model
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

# Load the BART tokenizer
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

# Preprocess the dataset
train_inputs = tokenizer.encode_plus(train_article, padding=True, truncation=True, return_tensors='pt')
train_ids = train_inputs['input_ids']
t_attention_mask = train_inputs['attention_mask']
train_labels = tokenizer.encode_plus(train_summary, padding=True, truncation=True, return_tensors='pt')

validation_inputs = tokenizer.encode_plus(validation_article, padding=True, truncation=True, return_tensors='pt')
validation_ids = validation_inputs['input_ids']
v_attention_mask = validation_inputs['attention_mask']
validation_labels = tokenizer.encode_plus(validation_summary, padding=True, truncation=True, return_tensors='pt')

#attention_mask = tokenizer.attention_mask(input_ids)

#inputs = tokenizer([data[0] for data in dataset], padding=True, truncation=True, return_tensors='pt')
#labels = tokenizer([data[1] for data in dataset], padding=True, truncation=True, return_tensors='pt')
#labels = tokenizer.encode_plus(train_summaryDataset, padding=True, truncation=True, return_tensors='pt')

# Fine-tune the BART model
optimizer = AdamW(model.parameters(), lr=1e-5)
for epoch in range(10):
    model.train()
    optimizer.zero_grad()
    #loss = model(input_ids['input_ids'], attention_mask=input_ids['attention_mask'], labels=labels['input_ids']).loss
    #loss = model(input_ids[:290], attention_mask[:290], labels=labels['input_ids'][:290]).loss
    loss = model(train_ids, t_attention_mask, labels=train_labels['input_ids']).loss
    loss.backward()
    optimizer.step()

    # Evaluate the model on a held-out validation set
    model.eval()
    with torch.no_grad():
        validation_loss = model(validation_ids, v_attention_mask, labels=validation_labels['input_ids']).loss

    print(f"Epoch {epoch + 1}: Train Loss={loss.item()}, Validation Loss={validation_loss.item()}")

# Save the trained model
model.save_pretrained("albanian-bart")

my_model = BartForConditionalGeneration.from_pretrained("/content/albanian-bart")
# Generate summaries using the fine-tuned BART model
text = 'Rama tha se ai është turpi më i madh që vendi ynë ka pas në krye, ndërsa shtoi se kush e voton atë turpëron veten, fëmijët, familjen, prindërit, të gjallë e të vdekur në shtëpinë e tij.Rama: Në 25 prill çoni në shtëpi Ilir Metën. Boll, mjaftë! Ilir Meta duhet të shkojë në shtëpi. Ilir Meta është turpi më i madh që Shqipëria ka mundur të ketë në krye, ai turp duhet zhbërë në 25 prill dhe turp kush voton Ilir Metën, turpëron veten, fëmijët, familjen, prindërit, të gjallë e të vdekur në shtëpinë e tij. Boll se nuk është Shqipëria peng i një njeriu që ka ikur dhe i ka lëshuar pëllumba dhe sorra bashkë.'
inputs = tokenizer([text], padding=True, truncation=True, return_tensors='pt')
summary_ids = my_model.generate(inputs['input_ids'], attention_mask=inputs['attention_mask'], max_length=150)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(summary)




/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1: Train Loss=9.027831077575684, Validation Loss=6.4243035316467285
Epoch 2: Train Loss=6.543473720550537, Validation Loss=5.136016845703125
Epoch 3: Train Loss=5.112931728363037, Validation Loss=3.6824965476989746
Epoch 4: Train Loss=3.9751741886138916, Validation Loss=2.060199737548828
Epoch 5: Train Loss=2.4078660011291504, Validation Loss=0.7157629132270813
Epoch 6: Train Loss=0.9441552758216858, Validation Loss=0.11691051721572876
Epoch 7: Train Loss=0.21503162384033203, Validation Loss=0.04731051251292229
Epoch 8: Train Loss=0.0723361000418663, Validation Loss=0.052778661251068115
Epoch 9: Train Loss=0.04584807530045509, Validation Loss=0.06116820126771927
Epoch 10: Train Loss=0.08566266298294067, Validation Loss=0.044926341623067856
Rama: Ilir Meta Ùstë turpi më i madh që vendi ynë ka pas në krye. Ai turp duhet zhbërë në 25 prill dhe turp kush voton Ilir Metën, turpëron veten, fëmijët, familjen, prindërit, të gjallë e të vdekur.


In [6]:
import csv
from csv import reader
from transformers import BartTokenizer, BartForConditionalGeneration
from nltk.translate.bleu_score import corpus_bleu
from rouge import Rouge


#  MODEL AND TOKENIZER IS GENERATED BEFORE

tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model = BartForConditionalGeneration.from_pretrained("/content/albanian-bart")
# Generate summaries for the validation data using the fine-tuned BART model
predicted_summaries = []
target_summaries = []
for article, summary in zip(test_article[:100], test_summary[:100]):
    #print("article")
    #print(example[0])
    #print("summary")
    #print(example[1])
    #input_ids = tokenizer.encode([example[0]], return_tensors="pt")
    #summary_ids = model.generate(input_ids)
    inputs = tokenizer([article], padding=True, truncation=True, return_tensors='pt')
    summary_ids = model.generate(inputs['input_ids'], attention_mask=inputs['attention_mask'], max_length=200)
    predicted_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    predicted_summaries.append(predicted_summary)
    target_summaries.append(summary)

#text = 'Kur kanë kaluar më shumë se 6 orë nga nisja e procesit të votimit në gjithë vendin, PD përmes zëdhënëses së fushatës Floriana Garo i bën thirrje qytetarëve që të dalin të votojnë."Po shohim një rritje të madhe të pjesëmarrjes në zgjedhje anekënd Shqipërisë. Kudo në Shqipëri qytetarët dalin të votojnë", tha ajo.Mesazhi i PD për shqiptarët, nga zëdhënësja e fushatës, Floriana Garo:Â "Po shohim një rritje të madhe të pjesëmarrjes në zgjedhje anekënd Shqipërisë. Kudo në Shqipëri qytetarët dalin të votojnë.Kjo është një ditë historike feste për demokracinë.U bëjmë thirrje të gjithëve të dalin dhe të votojnë "“ të mos e humbin mundësinë.Ky është momenti juaj. Ky është momenti i Shqipërisë. Dilni  dhe votoni!"'
#target_summary = 'Më shumë se 6 orë nga nisja e votimeve/ Mesazhi i PD për shqiptarët: Mos e humbisni mundësinë, dilni të votoni'
'''my_model = MBartForConditionalGeneration.from_pretrained("/content/albanian-bart")'''
'''text = 'Në një postim në Twitter, Rama tha se forcimi i institucioneve ligjzbatuese po nxjerr në pah edhe llumin e administratës publike.Sipas tij, fakti që këto raste bëhen publike tregon se mburoja e vjetër politike ka rënë. Ai shton se këtyre raste duhet ti shkohet deri në fund dhe forca e ligjit të mos ndalojë përpara askujt.Postimi i Edi Ramës në Twitter:Nuk e di kur do ta kuptojnë disa që Reforma në Drejtësi dhe forcimi i institucioneve ligjzbatuese bëhen pikërisht për të pastruar llumin! Prandaj kur del llumi falë punës së këtyre institucioneve, sduhet harruar se këto janë provat e vullnetit tonë kundër korrupsionit!E thënë ndryshe, në një kohë tjetër policia apo drejtësia nuk ia nxirrnin kurrë lakrat në shesh njerëzve të veshur me kostum zyrtar, ndërsa sot kohët ndryshojnë përditë për mirë edhe në këtë drejtim! Sepse sot vullneti politik është i plotë dhe mburoja e vjetër politike ka rënë!Unë uroj vetëm që tu shkohet sa më shumë deri në fund historive të ulëta me para të pista dhe forca e ligjit të mos ndalojë përpara askujt, kur faktet provojnë se është shkelur ligji për përfitime të paligjshme, nga kushdoqoftë.'
target_summary = 'Kryeministri Edi Rama ka reaguar për ngjarjen ku nënë e bir vodhën 120 mijë euro nga shtëpia e Alda Klosit, ish-drejtoreshë e kabinetit të ish-zv.ministrit Arben Ahmetaj. Vjedhja e nënë e bir tek shtëpia e zyrtares së Kryeministrisë/ Rama: Tu shkohet deri në fund historive të ulëta me para të pista!'
inputs = tokenizer([text], padding=True, truncation=True, return_tensors='pt')
summary_ids = model.generate(inputs['input_ids'], attention_mask=inputs['attention_mask'], max_length=200)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
target_summaries.append(target_summary)
predicted_summaries.append(summary)'''





ROUGE scores:
{'rouge-1': {'r': 0.1997374456241397, 'p': 0.2133513106219169, 'f': 0.20146649774397132}, 'rouge-2': {'r': 0.031201409853290354, 'p': 0.03384806870574309, 'f': 0.031667969941076235}, 'rouge-l': {'r': 0.16658838640148402, 'p': 0.17796409661745102, 'f': 0.1678989533742033}}


NameError: ignored

In [7]:
import numpy as np
#print(target_summary)
#print(summary)
# Compute the ROUGE metrics
rouge = Rouge()
rouge_scores = rouge.get_scores(predicted_summaries, target_summaries, avg=True)

print("ROUGE scores:")
print(rouge_scores)

score_1 = round(rouge_scores['rouge-1']['f'], 2)    
score_2 = round(rouge_scores['rouge-2']['f'], 2)    
score_L = round(rouge_scores['rouge-l']['f'], 2)    
print("rouge1:", score_1, "| rouge2:", score_2, "| rougeL:",
         score_2, "--> avg rouge:", round(np.mean(
         [score_1,score_2,score_L]), 2))

# Compute the BLEU metrics
bleu_scores = corpus_bleu([[summary] for summary in target_summaries], predicted_summaries)

print("BLEU score:")
print(bleu_scores)

ROUGE scores:
{'rouge-1': {'r': 0.1997374456241397, 'p': 0.2133513106219169, 'f': 0.20146649774397132}, 'rouge-2': {'r': 0.031201409853290354, 'p': 0.03384806870574309, 'f': 0.031667969941076235}, 'rouge-l': {'r': 0.16658838640148402, 'p': 0.17796409661745102, 'f': 0.1678989533742033}}
rouge1: 0.2 | rouge2: 0.03 | rougeL: 0.03 --> avg rouge: 0.13
BLEU score:
0.3255421919554536
